In [2]:
import pandas as pd
import numpy as np

In [5]:
events = pd.read_csv("../interm/doc_aux_train_doc_ids.csv.gz", index_col= False)

doc_events = events[["document_id"]].drop_duplicates()

In [7]:
doc_entities = pd.read_csv("../data/documents_entities.csv.gz", index_col=False)
doc_topics = pd.read_csv("../data/documents_topics.csv.gz", index_col=False)
doc_categories = pd.read_csv("../data/documents_categories.csv.gz", index_col=False)

In [8]:
doc_topics["topic_conf"] = doc_topics[["topic_id", "confidence_level"]].apply(tuple, axis=1)
doc_entities["entity_conf"] = doc_entities[["entity_id", "confidence_level"]].apply(tuple, axis=1)
doc_categories["categ_conf"] = doc_categories[["category_id", "confidence_level"]].apply(tuple, axis=1)

In [9]:
topics = doc_topics[["document_id", "topic_conf"]].groupby(["document_id"])["topic_conf"].apply(list).reset_index()

entities = doc_entities[["document_id", "entity_conf"]].groupby(["document_id"])["entity_conf"].apply(list).reset_index()

categories = doc_categories[["document_id", "categ_conf"]].groupby(["document_id"])["categ_conf"].apply(list).reset_index()

In [10]:
def getTopOne(l):
    if l == -99:
        return -99
    else:
        l = sorted(l, key = lambda x :x[1], reverse=True)
        return l[0][0]

In [11]:
def merge_select_topI(doc, aux_data, content,onKey = 'document_id'):
    doc_aux = doc.merge(aux_data, how = 'left', on = onKey)
    doc_aux.fillna(-99, inplace= True)
    doc_aux[content+ "_topI"] = doc_aux[content + "_conf"].apply(getTopOne)
    return doc_aux

In [12]:
selected_doc_topics = doc_events.merge(topics,how = 'left', on = 'document_id')

In [13]:
selected_doc_topics.fillna(-99,inplace= True)
selected_doc_topics.head()

,document_id,topic_conf
0,379743,"[(285.0, 0.0202538297549), (226.0, 0.019394842..."
1,1794259,"[(66.0, 0.0315848601509), (164.0, 0.0095238104..."
2,1179111,-99
3,1777797,"[(16.0, 0.0492014078746), (89.0, 0.03033053201..."
4,252458,"[(153.0, 0.100685146299), (265.0, 0.0661570192..."


In [14]:
selected_doc_topics["topic_topI"] = selected_doc_topics["topic_conf"].apply(getTopOne)

len(selected_doc_topics["topic_topI"].unique())

301

In [16]:
selected_doc_categ = merge_select_topI(selected_doc_topics, categories, content = "categ")

selected_doc_entity = merge_select_topI(selected_doc_categ, entities, content = "entity")

In [19]:
selected_doc_entity.to_csv("../interm/doc_topic_categ_entity_groupby_confidence_level_train.csv.gz",
                           index=False, compression="gzip")

selected_doc_entity[["document_id", "topic_topI", "entity_topI", "categ_topI"]]\
                    .to_csv("../interm/doc_aux_topI_topic_entity_categ_train.csv.gz", 
                            index=False, compression="gzip")

In [18]:
selected_doc_entity.head()

,document_id,topic_conf,topic_topI,categ_conf,categ_topI,entity_conf,entity_topI
0,379743,"[(285.0, 0.0202538297549), (226.0, 0.019394842...",285.0,"[(1203.0, 0.345951832), (1702.0, 0.026322422)]",1203.0,-99,-99
1,1794259,"[(66.0, 0.0315848601509), (164.0, 0.0095238104...",66.0,"[(1403.0, 0.353799984), (1702.0, 0.026919564)]",1403.0,"[(0db30daa7d11fbe9245e1bae8c4ef851, 0.69676003...",0db30daa7d11fbe9245e1bae8c4ef851
2,1179111,-99,-99.0,-99,-99.0,-99,-99
3,1777797,"[(16.0, 0.0492014078746), (89.0, 0.03033053201...",16.0,"[(1609.0, 0.494161164), (1403.0, 0.037599219)]",1609.0,-99,-99
4,252458,"[(153.0, 0.100685146299), (265.0, 0.0661570192...",153.0,"[(1403.0, 0.92), (1404.0, 0.07)]",1403.0,"[(97556fc7b4e378578a62e077b7eb1099, 0.95271146...",97556fc7b4e378578a62e077b7eb1099


In [135]:
len(selected_doc_entity["categ_topI"].unique())

91